## Grid Search

In [ ]:
from typing import Dict, List
from ddnn.nn import *
from ddnn.data import *
from ddnn.validation import *

In [ ]:
net = NeuralNetwork(
    [
        LinearLayer((8, 16)),
        ActivationFunction(),
    ]
)
estimator = Estimator(net) # fix to work without anything
grid = {}
grid["layers"] = [
    [(64, "ReLU"), (2, "linear")],
    [(32, "ReLU"), (32, "ReLU"), (2, "linear")]
]
grid["learning_rate"] = [1e-2, 6.6e-3, 3.3e-3, 1e-3]
grid["optimizer"] = ["Adam"]
grid["loss"] = ["MSE"]
grid["l2_coefficient"] = [1e-2, 6.6e-3, 3.3e-3, 1e-3]
grid["batchsize"] = [-1]
grid["weight_initializer"] = ["glorot_uniform"]
# grid['fan_mode'] = ['fan_in', 'fan_out']
selector = GridSearch(estimator, grid, 523552)
comb=1
for k,v in grid.items():
    comb*= len(v)
print(comb)

In [ ]:
data = read_ML_cup("train")
# data = read_monks(1, "train")
data, test = train_valid_split(data, seed=123)

In [ ]:
logger = Logger(estimator, ["MSE"], every=10)

In [ ]:
k_fold_results = selector.k_fold(
    data,
    5,
    n_epochs=1500,
    loss_list=["MEE", "MSE"],
    early_stopping=(5, 10, 1e-4),
    seed=123,
    on_fold_change=logger.update_fold,
    on_hp_change=logger.update_hp,
    training_callback=logger,
)

In [ ]:
results = {
    # "k_fold_results": k_fold_results
    "logs": logger._scores,
    "every": logger._every,
    "losses": logger._losses
}

In [ ]:
# save to file
import json
with open("trial.json", "w") as fp:
    json.dump(results, fp)

## Plotting

In [ ]:
# open with pandas
import pandas as pd
import json
with open("trial.json", "r") as fp:
    scores = json.load(fp)
df = pd.json_normalize(scores["logs"])
df["hp.layers"] = df["hp.layers"].astype(str) # convert layers to str repr

In [ ]:
from ipywidgets.widgets import interact_manual, interact
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
%matplotlib ipympl

# select loss, hyper get 2 plots one with train one with valid
fig, axs = plt.subplots(1,2, figsize=(10,4))

# hyperparameters in grid
params = {
    k: df[k].unique() 
    for k in df.columns[df.columns.str.startswith("hp.")]
}
# hyperparameters in grid with at least 2 values
fparams = {
    k: v 
    for k,v in params.items()
    if len(v) > 1
}

@interact(
    loss = scores["losses"],
    **fparams
)
def plot_results(loss, **kwargs):
    row_selection = pd.Series([True]*df.shape[0])
    for k,v in kwargs.items():
        row_selection &= (df[k] == v)
    rows = df[row_selection]

    # add parameter to title if missing
    for k in params:
        if k not in kwargs:
            kwargs[k] = params[k][0]

    fig.suptitle(", ".join([f"{k[3:]}={v}" for k,v in kwargs.items()]), wrap=True, fontsize=10)
    fig.tight_layout()
    for where, ax in zip(["train", "valid"], axs):
        col_selection = rows.columns.str.endswith(f"{where}.{loss}")
        data = rows[rows.columns[col_selection]]

        ys = [y for y in data.values][0]
        if loss != "binary_accuracy":
            bests = [min(y) for y in ys]
            form = "{:.2E}"
            logplot = True
        else:
            bests = [max(y) for y in ys]
            form = "{:.2}"
            logplot = False

        ax.clear()
        ax.set_title(where, fontsize=10)
        # scale to resemble number of epochs instead of plot points
        ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x*scores{"every"}))
        ax.xaxis.set_major_formatter(ticks_x)
        if logplot:
            ax.set_yscale("log")
        else:
            ax.set_yscale("linear")
        for y,best in zip(ys, bests):
            ax.plot(y, label=f"{form.format(best)}")
        ax.legend()

In [ ]:
results